In [1]:
##  LIBRERIAS  
import os
import numpy as np
import pandas as pd
import re
from unidecode import unidecode
from tqdm.notebook import trange, tqdm
import time
import math

import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point, LineString, Polygon

from mpl_toolkits import mplot3d
%matplotlib inline

from mpl_toolkits.mplot3d import Axes3D

pd.set_option('display.max_columns', 50)


## LECTURA ... Df Parametros
par_file_name="CLAG1.par"

#print(open(par_file_name, "r").read())
open(par_file_name, "r").close()
l=[re.split("[']", x) for x in open(par_file_name, "r")]
#print(x)
open(par_file_name, "r").close()            #cerrar archivo 
## diccionario de parámetros
param={}
param_l=[]
paran_v=[]

## OJO!! 
# Separador no es el mismo en todos los SO -->  Usar OS

## header:          PARÁMETRO 1
param[1]=l[0][0].strip()
param_l.append(l[0][1].strip().replace(os.sep,""))
## input file:          PARÁMETRO 2
param[2]=l[1][0].strip().rsplit(os.sep, 1)[-1]
param_l.append(l[1][1].strip().replace(os.sep,""))
## columns for data:          PARÁMETRO 3
param[3]=[int(x) for x in l[2][0].strip().split(",")]
param_l.append(l[2][1].strip().replace(os.sep,"")[0:16])
## initial search volume:          PARÁMETRO 4
param[4]=[float(x) for x in l[3][0].strip().split(",")]
param_l.append(l[3][1].strip().replace(os.sep,""))
## Number of Risk Thresholds:          PARÁMETRO 5
param[5]=[int(x) for x in l[4][0].strip().split(",")]
param_l.append(l[4][1].strip().replace(os.sep,"")[0:37])
## Selected risk thresholds:          PARÁMETRO 6
param[6]=[float(x) for x in l[5][0].strip().split(",")]
param_l.append(l[5][1].strip().replace(os.sep,""))
## Number of data in local volume:          PARÁMETRO 7
param[7]=[int(x) for x in l[6][0].strip().split(",")]
param_l.append(l[6][1].strip().replace(os.sep,""))
## Number of units:          PARÁMETRO 8
param[8]=[int(x) for x in l[7][0].strip().split(",")]
param_l.append(l[7][1].strip().replace(os.sep,""))
## Unit names:          PARÁMETRO 9
param[9]=[int(x) for x in l[8][0].strip().split(",")]
param_l.append(l[8][1].strip().replace(os.sep,""))
## not consider values less than:          PARÁMETRO 10
param[10]=[int(x) for x in l[9][0].strip().split(",")]
param_l.append(l[9][1].strip().replace(os.sep,""))
## output files name:          PARÁMETRO 11
param[11]=l[10][0].strip().rsplit(os.sep, 1)[-1]
param_l.append(l[10][1].strip().replace(os.sep,""))
## number of zones:          PARÁMETRO 12
param[12]=[int(x) for x in l[11][0].strip().split(",")]
param_l.append(l[11][1].strip().replace(os.sep,""))
## zone identifiers:          PARÁMETRO 13
param[13]=[int(x) for x in l[12][0].strip().split(",")]
param_l.append(l[12][1].strip().replace(os.sep,""))
## azimuth of search orientation per zone :          PARÁMETRO 14
param[14]=[int(x) for x in l[13][0].strip().split(",")]
param_l.append(l[13][1].strip().replace(os.sep,""))
## Anisotropy ratios:          PARÁMETRO 15
param[15]=[float(x) for x in l[14][0].strip().split(",")]
param_l.append(l[14][1].strip().replace(os.sep,""))
## Exclude data from the same drillhole:          PARÁMETRO 16
param[16]=[int(x) for x in l[15][0].strip().split(",")]
param_l.append(l[15][1].strip().replace(os.sep,""))
## Option to weight data by search expansion:          PARÁMETRO 17
param[17]=[int(x) for x in l[16][0].strip().split(",")]
param_l.append(l[16][1].strip().replace(os.sep,""))

df_parametros = pd.DataFrame(list(zip(list(range(1,18)), param_l, list(param.values()))),
                            columns=['Orden','Descripcion', 'Valor']
                        )


#df_parametros

df_CLAG_MR_2022 = pd.read_csv(param[2])
df_CLAG_MR_2022.head(2)
## Requerimiento: No NaN ni missing values
'''
Algoritmo localcapping no admite valores nulos ni missings
función missing_val(df) identifica casos missing y nulos
'''

if df_CLAG_MR_2022.isnull().any().any()==False:
    print("No hay valores nulos")
else:
    print("Hay valores nulos")


No hay valores nulos


In [2]:
df_CLAG_MR_2022

,COUNT,mid_x,mid_y,mid_z,Ag_ppm_BESTEL,NBKRT,AG_SUBZONAS,HOLEID
0,2336,4072.2950,4850.2450,4199.8669,25.146513,32,42,A019
1,2337,4073.5668,4848.7522,4196.7885,40.000000,31,42,A019
2,2338,4074.6121,4847.5252,4194.2584,35.799999,31,42,A019
3,2339,4075.6575,4846.2982,4191.7282,44.693333,31,42,A019
4,2340,4076.7028,4845.0713,4189.1980,45.193333,31,42,A019
...,...,...,...,...,...,...,...,...
4250,390969,4234.5329,5190.9378,4185.3258,7.000000,31,42,A913
4251,390970,4234.9222,5190.5647,4181.7917,7.457831,31,42,A913
4252,390971,4235.3126,5190.1954,4178.2573,8.333333,31,42,A913
4253,394849,3690.3063,4739.9698,4152.7452,122.400002,32,41,A951


In [3]:
float_dataframe_round=0

df_CLAG_MR_2022=df_CLAG_MR_2022.set_index('COUNT')
df_CLAG_MR_2022.iloc[:, :3]=df_CLAG_MR_2022.iloc[:, :3].round(float_dataframe_round)


df_CLAG_MR_2022["x_rot"]=0
df_CLAG_MR_2022["y_rot"]=0
df_CLAG_MR_2022["z_rot"]=0

df_CLAG_MR_2022["CAP1_P"]=df_CLAG_MR_2022.iloc[:,3]
df_CLAG_MR_2022["CPD1_P"]=0
df_CLAG_MR_2022["CAP2_P"]=df_CLAG_MR_2022.iloc[:,3]
df_CLAG_MR_2022["CPD2_P"]=0
df_CLAG_MR_2022["CAP3_P"]=df_CLAG_MR_2022.iloc[:,3]
df_CLAG_MR_2022["CPD3_P"]=0
df_CLAG_MR_2022["CAP4_P"]=df_CLAG_MR_2022.iloc[:,3]
df_CLAG_MR_2022["CPD4_P"]=0
df_CLAG_MR_2022["CAP5_P"]=df_CLAG_MR_2022.iloc[:,3]
df_CLAG_MR_2022["CPD5_P"]=0
df_CLAG_MR_2022["CAP6_P"]=df_CLAG_MR_2022.iloc[:,3]
df_CLAG_MR_2022["CPD6_P"]=0

df_CLAG_MR_2022["EVAL_P"]=0

df_CLAG_MR_2022["NEXP_P"]=0
df_CLAG_MR_2022["LEN_NT_P"]=0
df_CLAG_MR_2022["LEN_TO_P"]=0

df_CLAG_MR_2022["SUM_SN_P"]=0
df_CLAG_MR_2022["SUM_CN_P"]=0

df_CLAG_MR_2022["AVG_SN_P"]=0
df_CLAG_MR_2022["AVG_CN_P"]=0

df_CLAG_MR_2022["PROP_SN_P"]=0
df_CLAG_MR_2022["PROP_CN_P"]=0





In [4]:
df_CLAG_MR_2022.head(2)

,mid_x,mid_y,mid_z,Ag_ppm_BESTEL,NBKRT,AG_SUBZONAS,HOLEID,x_rot,y_rot,z_rot,CAP1_P,CPD1_P,CAP2_P,CPD2_P,CAP3_P,CPD3_P,CAP4_P,CPD4_P,CAP5_P,CPD5_P,CAP6_P,CPD6_P,EVAL_P,NEXP_P,LEN_NT_P,LEN_TO_P,SUM_SN_P,SUM_CN_P,AVG_SN_P,AVG_CN_P,PROP_SN_P,PROP_CN_P
COUNT,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2336,4072.0,4850.0,4200.0,25.146513,32,42,A019,0,0,0,25.146513,0,25.146513,0,25.146513,0,25.146513,0,25.146513,0,25.146513,0,0,0,0,0,0,0,0,0,0,0
2337,4074.0,4849.0,4197.0,40.000000,31,42,A019,0,0,0,40.000000,0,40.000000,0,40.000000,0,40.000000,0,40.000000,0,40.000000,0,0,0,0,0,0,0,0,0,0,0


In [20]:

zonas = df_parametros.Valor[12]
rocks=df_parametros.Valor[8]
iterador=0      ##  PARA SIGNAR RATIO x ZONA

expansiones_max=10
parametro_expansion=0.25
float_hemi_accuracy=2

float_dataframe_round=2     ## redondeo en dataframe

flag_drillhole=df_parametros.Valor[15][0] ### = 1 EXCLUDE SAME DRILLHOLE

freno_1_composito_x_roca=0  ###  SOLO LEER UN SOLO COMPOSITO POR COMBINACION ROCA/ZONA
freno_2_roca_x_zona=0  ###  SOLO LEER UNA SOLA ROCA POR ZONA
freno_3_zona=0    ### SOLO LEER UNICA ZONA
hemi_axis_x, hemi_axis_y, hemi_axis_z = [], [], []
axis_x, axis_y, axis_z = [], [], []


elipsoide_on=0     # activar o desactivar elipsoide,  en defecto activa paralelepipedo


for zone in zonas:  
    print("\n")
    print("zone:",zone)
    print(" ")  
    azimuth=df_parametros.Valor[13][iterador]
    print(" azimuth:",azimuth)
    if azimuth < 90:
        cartesiana=90-azimuth 
    elif azimuth < 180:
        cartesiana=360-(azimuth-90) 
    elif azimuth < 270:
        cartesiana=270-(azimuth-180) 
    elif azimuth < 360:
        cartesiana=180-(azimuth-270) 
    print(" cartesiana:",cartesiana)
    ratio=df_parametros.Valor[14][iterador]
    print(" ratio anisotropia:",ratio)
    

    ###
    # PARAMETROS EXPANSION    ELIPSOIDE
    for i in range(expansiones_max):       #  30*30*30       30*75*30
        # print((df_parametros.Valor[3][0] / 2)*((1+parametro_expansion)**i))
        # print(((df_parametros.Valor[3][1]*ratio) / 2)*((1+parametro_expansion)**i))
        # print((df_parametros.Valor[3][2] / 2)*((1+parametro_expansion)**i))
        # print((df_parametros.Valor[3][0] / 2)*((1+parametro_expansion)**i)*2)
        # print(((df_parametros.Valor[3][1]*ratio) / 2)*((1+parametro_expansion)**i)*2)
        # print((df_parametros.Valor[3][2] / 2)*((1+parametro_expansion)**i)*2)
        # print(" ")
        semi_x=(df_parametros.Valor[3][0] / 2)*((1+parametro_expansion)**i)
        semi_y=((df_parametros.Valor[3][1]*ratio) / 2)*((1+parametro_expansion)**i)
        semi_z=(df_parametros.Valor[3][2] / 2)*((1+parametro_expansion)**i)
        hemi_axis_x.append(round(semi_x,float_hemi_accuracy))
        hemi_axis_y.append(round(semi_y,float_hemi_accuracy))
        hemi_axis_z.append(round(semi_z,float_hemi_accuracy))
        axis_x.append(round(semi_x*2,float_hemi_accuracy))
        axis_y.append(round(semi_y*2,float_hemi_accuracy))
        axis_z.append(round(semi_z*2,float_hemi_accuracy))
        #print(semi_x*2,semi_y*2,semi_z*2)
        # print(hemi_axis_x,hemi_axis_y,hemi_axis_z)
        # print(" ")
    print(" hemi_exis:",hemi_axis_x,hemi_axis_y,hemi_axis_z)
    ###

    
    for rock in rocks:
        print("\n")
        print("    rock:",rock)
        print("    zone:",zone)
        print('\n')


        theta = np.deg2rad(cartesiana) 
        rot_matrix = np.array([
            [np.cos(theta), -np.sin(theta), 0],
            [np.sin(theta), np.cos(theta), 0],
            [0, 0, 1]])
        
        
        df=df_CLAG_MR_2022[(df_CLAG_MR_2022.iloc[:, 4] == rock) & (df_CLAG_MR_2022.iloc[:, 5] == zone)]

        for i, row in df.iterrows():
            rotated_coords = np.dot([row['mid_x'], row['mid_y'], row['mid_z']], rot_matrix)
            
            df_CLAG_MR_2022.at[i, 'x_rot'] = rotated_coords[0]
            df_CLAG_MR_2022.at[i, 'y_rot'] = rotated_coords[1]
            df_CLAG_MR_2022.at[i, 'z_rot'] = rotated_coords[2]


        
        
        for index, row in df.iterrows():
            x=row['x_rot']
            y=row['y_rot']
            z=row['z_rot']

            xr=row['mid_x']
            yr=row['mid_y']
            zr=row['mid_z']
            
            hole=row.iloc[6]
            ley=row.iloc[3]
            print("        ID:", index)
            # print("         x:", x)
            # print("         y:", y)
            # print("         z:", z)
            # print("         xr:", xr)
            # print("         yr:", yr)
            # print("         zr:", zr)
            
            # print("         hole:", hole)
            # print("         ley:", ley)
            # print(" ")

            index_inner=[]
            ley_inner=[]
            hole_inner=[]

            for i in range(expansiones_max):
                # print(" ")
                # print("        expansion: ",i)
                # print(" ")

                # print(hemi_axis_x[i],
                #       hemi_axis_y[i],
                #       hemi_axis_z[i])
                
                a=hemi_axis_x[i]
                b=hemi_axis_y[i]
                c=hemi_axis_z[i]

                aa=axis_x[i]
                bb=axis_y[i]
                cc=axis_z[i]

                m_ai=x-a
                m_as=x+a
                m_bi=y-b
                m_bs=y+b
                m_ci=z-c
                m_cs=z+c

                for subindex, subrow in df.iterrows():
                    sx=subrow['x_rot']
                    sy=subrow['y_rot']
                    sz=subrow['z_rot']
                    # print("            ID sub:", subindex)
                    # print("             x sub:", subrow['x_rot'])
                    # print("             y sub:", subrow['y_rot'])
                    # print("             z sub:", subrow['z_rot'])
                    # print("             hole sub:", subrow.iloc[6])
                    # print("             ley sub:", subrow.iloc[3])
                    # print(" ")

                    if elipsoide_on==1:
                        if (((sx-x)**2)/(aa**2) + ((sy-y)**2)/(bb**2) + ((sz-z)**2)/(cc**2)) <= 1:
                            #print("             dentro de elipsode")
                            if flag_drillhole==1:
                                #print("             flag_drillhole:",flag_drillhole)
                                if hole!=subrow.iloc[6]:
                                    if subindex not in index_inner:
                                        index_inner.append(subindex)
                            else:
                                if subindex not in index_inner:
                                    index_inner.append(subindex)

                    else:
                        if m_ai < subrow['x_rot'] < m_as and m_bi < subrow['y_rot'] < m_bs and m_ci < subrow['z_rot'] < m_cs:
                            #print("             dentro de cubo")
                            if flag_drillhole==1:
                                #print("             flag_drillhole:",flag_drillhole)
                                if hole!=subrow.iloc[6]:
                                    if subindex not in index_inner:
                                        index_inner.append(subindex)
                            else:
                                if subindex not in index_inner:
                                    index_inner.append(subindex)

                if len(index_inner) > 20:   
                    # print(" ")
                    # print("             index_inner:",index_inner)
                    # print("             len index_inner:",len(index_inner))         
                    # print("##")
                    break    
            
            # print(" ")
            # print(" row", row)
            ###########################  MODIF
            df_CLAG_MR_2022.loc[index, df_CLAG_MR_2022.columns[22]] = 99  # EVAL_P
            df_CLAG_MR_2022.loc[index, df_CLAG_MR_2022.columns[23]] = i  # N EXPAN
            df_CLAG_MR_2022.loc[index, df_CLAG_MR_2022.columns[24]] = len(index_inner)  # N PUNTOS
                      
            print(" ")
            print(" ")

            inner_df = df.loc[index_inner]
            inner_df['dist'] = np.sqrt((inner_df['x_rot'] - x)**2 + (inner_df['y_rot'] - y)**2 + (inner_df['z_rot'] - z)**2)
            inner_df = inner_df.sort_values(by=['dist'])
            try:
                inner_df = inner_df.iloc[:20]
            except:
                inner_df = inner_df

            # print("$$")
            # print(len(index_inner))
            # print(inner_df)
            # print(" ")

            df_CLAG_MR_2022.loc[index, df_CLAG_MR_2022.columns[25]] = len(inner_df)  #  DATOS EXPANDIDOS TOP

            sumwo = inner_df.iloc[:, 3].sum()
            sumwt = inner_df.iloc[:, 3].sum()+ley

            avgwo= sumwo/len(inner_df)
            avgwt= sumwt/(len(inner_df)+1)

            propwo=ley/sumwo
            propwt=ley/sumwt

            # print( "sumwo:",sumwo)
            # print( "ley:",ley)
            # print( "sumwt:",sumwt)
            # print( "len df:",len(index_inner))
            # print( "avgwo:",avgwo)
            # print( "avgwt:",avgwt)
            # print( "propwo:",propwo)
            # print( "propwt:",propwt)

            # print( "expandido_notop:",len(index_inner))
            # print( "expandido_top:",len(inner_df))

            # print( "expansiones:",i)            

            df_CLAG_MR_2022.loc[index, df_CLAG_MR_2022.columns[26]] = sumwo  #  SUMA WO
            df_CLAG_MR_2022.loc[index, df_CLAG_MR_2022.columns[27]] = sumwt  #  SUMA WT
            df_CLAG_MR_2022.loc[index, df_CLAG_MR_2022.columns[28]] = avgwo  #  AVG WO
            df_CLAG_MR_2022.loc[index, df_CLAG_MR_2022.columns[29]] = avgwt  #  AVG WT
            df_CLAG_MR_2022.loc[index, df_CLAG_MR_2022.columns[30]] = propwo  #  PROP WO
            df_CLAG_MR_2022.loc[index, df_CLAG_MR_2022.columns[31]] = propwt  #  PROP WT


            variax=abs((avgwt/avgwo)-1)
            print("        variax:",variax)


            if variax > df_parametros.Valor[5][0]:
                print("         T1")
                df_CLAG_MR_2022.loc[index, df_CLAG_MR_2022.columns[8]] = avgwt + (ley*df_parametros.Valor[5][0])  
                df_CLAG_MR_2022.loc[index, df_CLAG_MR_2022.columns[9]] = 9  
                if variax > df_parametros.Valor[5][1]:
                    print("         T2")
                    df_CLAG_MR_2022.loc[index, df_CLAG_MR_2022.columns[10]] = avgwt + (ley*df_parametros.Valor[5][1])  
                    df_CLAG_MR_2022.loc[index, df_CLAG_MR_2022.columns[11]] = 9  
                    if variax > df_parametros.Valor[5][2]:
                        print("         T3")
                        df_CLAG_MR_2022.loc[index, df_CLAG_MR_2022.columns[12]] =  avgwt + (ley*df_parametros.Valor[5][2]) 
                        df_CLAG_MR_2022.loc[index, df_CLAG_MR_2022.columns[13]] = 9  
                        if variax > df_parametros.Valor[5][3]:
                            print("         T4")
                            df_CLAG_MR_2022.loc[index, df_CLAG_MR_2022.columns[14]] =  avgwt + (ley*df_parametros.Valor[5][3]) 
                            df_CLAG_MR_2022.loc[index, df_CLAG_MR_2022.columns[15]] = 9  
                            if variax > df_parametros.Valor[5][4]:
                                print("         T5")
                                df_CLAG_MR_2022.loc[index, df_CLAG_MR_2022.columns[16]] = avgwt + (ley*df_parametros.Valor[5][4])  
                                df_CLAG_MR_2022.loc[index, df_CLAG_MR_2022.columns[17]] = 9  
                                if variax > df_parametros.Valor[5][5]: 
                                    print("         T6")
                                    df_CLAG_MR_2022.loc[index, df_CLAG_MR_2022.columns[18]] = avgwt + (ley*df_parametros.Valor[5][5])  
                                    df_CLAG_MR_2022.loc[index, df_CLAG_MR_2022.columns[19]] = 9  
                                    pass


            # print(" ")
            # print(" ")


            freno_1_composito_x_roca=freno_1_composito_x_roca+1
            if freno_1_composito_x_roca>100:
                break
        freno_2_roca_x_zona=freno_2_roca_x_zona+1
        if freno_2_roca_x_zona>0:
            break
    freno_3_zona=freno_3_zona+1
    if freno_3_zona>0:
        break
    iterador=iterador+1


    print(" ")
    print(" ")






zone: 41
 
 azimuth: 140
 cartesiana: 310
 ratio anisotropia: 2.5
 hemi_exis: [15.0, 18.75, 23.44, 29.3, 36.62, 45.78, 57.22, 71.53, 89.41, 111.76] [37.5, 46.88, 58.59, 73.24, 91.55, 114.44, 143.05, 178.81, 223.52, 279.4] [15.0, 18.75, 23.44, 29.3, 36.62, 45.78, 57.22, 71.53, 89.41, 111.76]


    rock: 31
    zone: 41


        ID: 6586
 
 
        variax: 0.04413978961208742
        ID: 6587
 
 
        variax: 0.03922083429282652
        ID: 6588
 
 
        variax: 0.004788159655320312
        ID: 6589
 
 
        variax: 0.017646349335223577
        ID: 6590
 
 
        variax: 0.011559567699749529
        ID: 7164
 
 
        variax: 0.01525878740870934
        ID: 7173
 
 
        variax: 0.00827660589328949
        ID: 7174
 
 
        variax: 0.012027897002396593
        ID: 7175
 
 
        variax: 0.0035163640028335053
        ID: 7176
 
 
        variax: 0.01873595133898709
        ID: 7177
 
 
        variax: 0.02058175309313537
        ID: 8060
 
 
        variax: 0.0095

In [21]:
df_CLAG_MR_2022['ID'] = df_CLAG_MR_2022.index
df_CLAG_MR_2022

,mid_x,mid_y,mid_z,Ag_ppm_BESTEL,NBKRT,AG_SUBZONAS,HOLEID,x_rot,y_rot,z_rot,CAP1_P,CPD1_P,CAP2_P,CPD2_P,CAP3_P,CPD3_P,CAP4_P,CPD4_P,CAP5_P,CPD5_P,CAP6_P,CPD6_P,EVAL_P,NEXP_P,LEN_NT_P,LEN_TO_P,SUM_SN_P,SUM_CN_P,AVG_SN_P,AVG_CN_P,PROP_SN_P,PROP_CN_P,ID
COUNT,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2336,4072.0,4850.0,4200.0,25.146513,32,42,A019,0.0,0.0,0,25.146513,0,25.146513,0,25.146513,0,25.146513,0,25.146513,0,25.146513,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,2336
2337,4074.0,4849.0,4197.0,40.000000,31,42,A019,0.0,0.0,0,40.000000,0,40.000000,0,40.000000,0,40.000000,0,40.000000,0,40.000000,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,2337
2338,4075.0,4848.0,4194.0,35.799999,31,42,A019,0.0,0.0,0,35.799999,0,35.799999,0,35.799999,0,35.799999,0,35.799999,0,35.799999,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,2338
2339,4076.0,4846.0,4192.0,44.693333,31,42,A019,0.0,0.0,0,44.693333,0,44.693333,0,44.693333,0,44.693333,0,44.693333,0,44.693333,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,2339
2340,4077.0,4845.0,4189.0,45.193333,31,42,A019,0.0,0.0,0,45.193333,0,45.193333,0,45.193333,0,45.193333,0,45.193333,0,45.193333,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,2340
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390969,4235.0,5191.0,4185.0,7.000000,31,42,A913,0.0,0.0,0,7.000000,0,7.000000,0,7.000000,0,7.000000,0,7.000000,0,7.000000,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,390969
390970,4235.0,5191.0,4182.0,7.457831,31,42,A913,0.0,0.0,0,7.457831,0,7.457831,0,7.457831,0,7.457831,0,7.457831,0,7.457831,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,390970
390971,4235.0,5190.0,4178.0,8.333333,31,42,A913,0.0,0.0,0,8.333333,0,8.333333,0,8.333333,0,8.333333,0,8.333333,0,8.333333,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,390971


In [22]:
dx=df_CLAG_MR_2022.iloc[:, list(range(10,22)) + [32]]
dx

,CAP1_P,CPD1_P,CAP2_P,CPD2_P,CAP3_P,CPD3_P,CAP4_P,CPD4_P,CAP5_P,CPD5_P,CAP6_P,CPD6_P,ID
COUNT,,,,,,,,,,,,,
2336,25.146513,0,25.146513,0,25.146513,0,25.146513,0,25.146513,0,25.146513,0,2336
2337,40.000000,0,40.000000,0,40.000000,0,40.000000,0,40.000000,0,40.000000,0,2337
2338,35.799999,0,35.799999,0,35.799999,0,35.799999,0,35.799999,0,35.799999,0,2338
2339,44.693333,0,44.693333,0,44.693333,0,44.693333,0,44.693333,0,44.693333,0,2339
2340,45.193333,0,45.193333,0,45.193333,0,45.193333,0,45.193333,0,45.193333,0,2340
...,...,...,...,...,...,...,...,...,...,...,...,...,...
390969,7.000000,0,7.000000,0,7.000000,0,7.000000,0,7.000000,0,7.000000,0,390969
390970,7.457831,0,7.457831,0,7.457831,0,7.457831,0,7.457831,0,7.457831,0,390970
390971,8.333333,0,8.333333,0,8.333333,0,8.333333,0,8.333333,0,8.333333,0,390971


In [23]:
ag_df=pd.read_csv("C:\FORTRAN\clsilver_mr20221.out",sep=",")
ag_df

,ID,BHID,X,Y,Z,CODE,ZONE,VALUE,CAPVAL1,CAPPED1,CAPVAL2,CAPPED2,CAPVAL3,CAPPED3,CAPVAL4,CAPPED4,CAPVAL5,CAPPED5,CAPVAL6,CAPPED6,AvgSin,PROP_12,Num_Data
0,2336,A019,4072.29,4850.25,4199.87,32,42.0,25.14651,25.14651,0,25.14651,0,25.14651,0,25.14651,0,25.14651,0,25.14651,0,30.6503,0.231,21
1,2337,A019,4073.57,4848.75,4196.79,31,42.0,40.00000,40.00000,0,40.00000,0,40.00000,0,40.00000,0,40.00000,0,40.00000,0,25.7654,0.213,21
2,2338,A019,4074.61,4847.53,4194.26,31,42.0,35.80000,35.80000,0,35.80000,0,35.80000,0,35.80000,0,35.80000,0,35.80000,0,26.4273,0.196,21
3,2339,A019,4075.66,4846.30,4191.73,31,42.0,44.69333,44.69333,0,44.69333,0,44.69333,0,44.69333,0,44.69333,0,44.69333,0,25.8114,0.179,21
4,2340,A019,4076.70,4845.07,4189.20,31,42.0,45.19333,45.19333,0,45.19333,0,45.19333,0,45.19333,0,45.19333,0,45.19333,0,25.5051,0.176,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4250,390969,A913,4234.53,5190.94,4185.33,31,42.0,7.00000,7.00000,0,7.00000,0,7.00000,0,7.00000,0,7.00000,0,7.00000,0,15.1952,0.125,21
4251,390970,A913,4234.92,5190.56,4181.79,31,42.0,7.45783,7.45783,0,7.45783,0,7.45783,0,7.45783,0,7.45783,0,7.45783,0,15.1952,0.119,21
4252,390971,A913,4235.31,5190.20,4178.26,31,42.0,8.33333,8.33333,0,8.33333,0,8.33333,0,8.33333,0,8.33333,0,8.33333,0,15.1952,0.163,21
4253,394849,A951,3690.31,4739.97,4152.75,32,41.0,122.40000,122.40000,0,122.40000,0,122.40000,0,122.40000,0,122.40000,0,122.40000,0,46.7659,0.000,21


In [24]:
df_merged = pd.merge(df_CLAG_MR_2022, ag_df, on='ID', how='outer')
df_merged

,mid_x,mid_y,mid_z,Ag_ppm_BESTEL,NBKRT,AG_SUBZONAS,HOLEID,x_rot,y_rot,z_rot,CAP1_P,CPD1_P,CAP2_P,CPD2_P,CAP3_P,CPD3_P,CAP4_P,CPD4_P,CAP5_P,CPD5_P,CAP6_P,CPD6_P,EVAL_P,NEXP_P,LEN_NT_P,...,PROP_SN_P,PROP_CN_P,ID,BHID,X,Y,Z,CODE,ZONE,VALUE,CAPVAL1,CAPPED1,CAPVAL2,CAPPED2,CAPVAL3,CAPPED3,CAPVAL4,CAPPED4,CAPVAL5,CAPPED5,CAPVAL6,CAPPED6,AvgSin,PROP_12,Num_Data
0,4072.0,4850.0,4200.0,25.146513,32,42,A019,0.0,0.0,0,25.146513,0,25.146513,0,25.146513,0,25.146513,0,25.146513,0,25.146513,0,0,0,0,...,0.0,0.0,2336,A019,4072.29,4850.25,4199.87,32,42.0,25.14651,25.14651,0,25.14651,0,25.14651,0,25.14651,0,25.14651,0,25.14651,0,30.6503,0.231,21
1,4074.0,4849.0,4197.0,40.000000,31,42,A019,0.0,0.0,0,40.000000,0,40.000000,0,40.000000,0,40.000000,0,40.000000,0,40.000000,0,0,0,0,...,0.0,0.0,2337,A019,4073.57,4848.75,4196.79,31,42.0,40.00000,40.00000,0,40.00000,0,40.00000,0,40.00000,0,40.00000,0,40.00000,0,25.7654,0.213,21
2,4075.0,4848.0,4194.0,35.799999,31,42,A019,0.0,0.0,0,35.799999,0,35.799999,0,35.799999,0,35.799999,0,35.799999,0,35.799999,0,0,0,0,...,0.0,0.0,2338,A019,4074.61,4847.53,4194.26,31,42.0,35.80000,35.80000,0,35.80000,0,35.80000,0,35.80000,0,35.80000,0,35.80000,0,26.4273,0.196,21
3,4076.0,4846.0,4192.0,44.693333,31,42,A019,0.0,0.0,0,44.693333,0,44.693333,0,44.693333,0,44.693333,0,44.693333,0,44.693333,0,0,0,0,...,0.0,0.0,2339,A019,4075.66,4846.30,4191.73,31,42.0,44.69333,44.69333,0,44.69333,0,44.69333,0,44.69333,0,44.69333,0,44.69333,0,25.8114,0.179,21
4,4077.0,4845.0,4189.0,45.193333,31,42,A019,0.0,0.0,0,45.193333,0,45.193333,0,45.193333,0,45.193333,0,45.193333,0,45.193333,0,0,0,0,...,0.0,0.0,2340,A019,4076.70,4845.07,4189.20,31,42.0,45.19333,45.19333,0,45.19333,0,45.19333,0,45.19333,0,45.19333,0,45.19333,0,25.5051,0.176,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4250,4235.0,5191.0,4185.0,7.000000,31,42,A913,0.0,0.0,0,7.000000,0,7.000000,0,7.000000,0,7.000000,0,7.000000,0,7.000000,0,0,0,0,...,0.0,0.0,390969,A913,4234.53,5190.94,4185.33,31,42.0,7.00000,7.00000,0,7.00000,0,7.00000,0,7.00000,0,7.00000,0,7.00000,0,15.1952,0.125,21
4251,4235.0,5191.0,4182.0,7.457831,31,42,A913,0.0,0.0,0,7.457831,0,7.457831,0,7.457831,0,7.457831,0,7.457831,0,7.457831,0,0,0,0,...,0.0,0.0,390970,A913,4234.92,5190.56,4181.79,31,42.0,7.45783,7.45783,0,7.45783,0,7.45783,0,7.45783,0,7.45783,0,7.45783,0,15.1952,0.119,21
4252,4235.0,5190.0,4178.0,8.333333,31,42,A913,0.0,0.0,0,8.333333,0,8.333333,0,8.333333,0,8.333333,0,8.333333,0,8.333333,0,0,0,0,...,0.0,0.0,390971,A913,4235.31,5190.20,4178.26,31,42.0,8.33333,8.33333,0,8.33333,0,8.33333,0,8.33333,0,8.33333,0,8.33333,0,15.1952,0.163,21
4253,3690.0,4740.0,4153.0,122.400002,32,41,A951,0.0,0.0,0,122.400002,0,122.400002,0,122.400002,0,122.400002,0,122.400002,0,122.400002,0,0,0,0,...,0.0,0.0,394849,A951,3690.31,4739.97,4152.75,32,41.0,122.40000,122.40000,0,122.40000,0,122.40000,0,122.40000,0,122.40000,0,122.40000,0,46.7659,0.000,21


In [65]:
df_merged.columns

Index(['mid_x', 'mid_y', 'mid_z', 'Ag_ppm_BESTEL', 'NBKRT', 'AG_SUBZONAS',
       'HOLEID', 'x_rot', 'y_rot', 'z_rot', 'CAP1_P', 'CPD1_P', 'CAP2_P',
       'CPD2_P', 'CAP3_P', 'CPD3_P', 'CAP4_P', 'CPD4_P', 'CAP5_P', 'CPD5_P',
       'CAP6_P', 'CPD6_P', 'EVAL_P', 'NEXP_P', 'LEN_NT_P', 'LEN_TO_P',
       'SUM_SN_P', 'SUM_CN_P', 'AVG_SN_P', 'AVG_CN_P', 'PROP_SN_P',
       'PROP_CN_P', 'ID', 'BHID', 'X', 'Y', 'Z', 'CODE', 'ZONE', 'VALUE',
       'CAPVAL1', 'CAPPED1', 'CAPVAL2', 'CAPPED2', 'CAPVAL3', 'CAPPED3',
       'CAPVAL4', 'CAPPED4', 'CAPVAL5', 'CAPPED5', 'CAPVAL6', 'CAPPED6',
       'AvgSin', 'PROP_12', 'Num_Data'],
      dtype='object')

In [66]:
df_nuevo = df_merged.loc[:, ['EVAL_P',
                             'ID',
                             'Ag_ppm_BESTEL','VALUE',
                             'CAP1_P', 'CPD1_P',
                             'CAPVAL1', 'CAPPED1',
                             'CAP2_P', 'CPD2_P',
                             'CAPVAL2', 'CAPPED2',
                             'CAP3_P', 'CPD3_P',
                             'CAPVAL3', 'CAPPED3',
                             'CAP4_P', 'CPD4_P',
                             'CAPVAL4', 'CAPPED4',
                             'CAP5_P', 'CPD5_P',
                             'CAPVAL5', 'CAPPED5',
                             'CAP6_P', 'CPD6_P',
                             'CAPVAL6', 'CAPPED6',
                             'SUM_SN_P', 'SUM_CN_P', 'AVG_SN_P', 'AVG_CN_P', 'PROP_SN_P', 'PROP_CN_P',
                             'AvgSin', 'PROP_12'                                                                                  
                             ]]

In [67]:
df_nuevo

,EVAL_P,ID,Ag_ppm_BESTEL,VALUE,CAP1_P,CPD1_P,CAPVAL1,CAPPED1,CAP2_P,CPD2_P,CAPVAL2,CAPPED2,CAP3_P,CPD3_P,CAPVAL3,CAPPED3,CAP4_P,CPD4_P,CAPVAL4,CAPPED4,CAP5_P,CPD5_P,CAPVAL5,CAPPED5,CAP6_P,CPD6_P,CAPVAL6,CAPPED6,SUM_SN_P,SUM_CN_P,AVG_SN_P,AVG_CN_P,PROP_SN_P,PROP_CN_P,AvgSin,PROP_12
0,0,2336,25.146513,25.14651,25.146513,0,25.14651,0,25.146513,0,25.14651,0,25.146513,0,25.14651,0,25.146513,0,25.14651,0,25.146513,0,25.14651,0,25.146513,0,25.14651,0,0.0,0.0,0.0,0.0,0.0,0.0,30.6503,0.231
1,0,2337,40.000000,40.00000,40.000000,0,40.00000,0,40.000000,0,40.00000,0,40.000000,0,40.00000,0,40.000000,0,40.00000,0,40.000000,0,40.00000,0,40.000000,0,40.00000,0,0.0,0.0,0.0,0.0,0.0,0.0,25.7654,0.213
2,0,2338,35.799999,35.80000,35.799999,0,35.80000,0,35.799999,0,35.80000,0,35.799999,0,35.80000,0,35.799999,0,35.80000,0,35.799999,0,35.80000,0,35.799999,0,35.80000,0,0.0,0.0,0.0,0.0,0.0,0.0,26.4273,0.196
3,0,2339,44.693333,44.69333,44.693333,0,44.69333,0,44.693333,0,44.69333,0,44.693333,0,44.69333,0,44.693333,0,44.69333,0,44.693333,0,44.69333,0,44.693333,0,44.69333,0,0.0,0.0,0.0,0.0,0.0,0.0,25.8114,0.179
4,0,2340,45.193333,45.19333,45.193333,0,45.19333,0,45.193333,0,45.19333,0,45.193333,0,45.19333,0,45.193333,0,45.19333,0,45.193333,0,45.19333,0,45.193333,0,45.19333,0,0.0,0.0,0.0,0.0,0.0,0.0,25.5051,0.176
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4250,0,390969,7.000000,7.00000,7.000000,0,7.00000,0,7.000000,0,7.00000,0,7.000000,0,7.00000,0,7.000000,0,7.00000,0,7.000000,0,7.00000,0,7.000000,0,7.00000,0,0.0,0.0,0.0,0.0,0.0,0.0,15.1952,0.125
4251,0,390970,7.457831,7.45783,7.457831,0,7.45783,0,7.457831,0,7.45783,0,7.457831,0,7.45783,0,7.457831,0,7.45783,0,7.457831,0,7.45783,0,7.457831,0,7.45783,0,0.0,0.0,0.0,0.0,0.0,0.0,15.1952,0.119
4252,0,390971,8.333333,8.33333,8.333333,0,8.33333,0,8.333333,0,8.33333,0,8.333333,0,8.33333,0,8.333333,0,8.33333,0,8.333333,0,8.33333,0,8.333333,0,8.33333,0,0.0,0.0,0.0,0.0,0.0,0.0,15.1952,0.163
4253,0,394849,122.400002,122.40000,122.400002,0,122.40000,0,122.400002,0,122.40000,0,122.400002,0,122.40000,0,122.400002,0,122.40000,0,122.400002,0,122.40000,0,122.400002,0,122.40000,0,0.0,0.0,0.0,0.0,0.0,0.0,46.7659,0.000


In [68]:
df=df_nuevo
df

,EVAL_P,ID,Ag_ppm_BESTEL,VALUE,CAP1_P,CPD1_P,CAPVAL1,CAPPED1,CAP2_P,CPD2_P,CAPVAL2,CAPPED2,CAP3_P,CPD3_P,CAPVAL3,CAPPED3,CAP4_P,CPD4_P,CAPVAL4,CAPPED4,CAP5_P,CPD5_P,CAPVAL5,CAPPED5,CAP6_P,CPD6_P,CAPVAL6,CAPPED6,SUM_SN_P,SUM_CN_P,AVG_SN_P,AVG_CN_P,PROP_SN_P,PROP_CN_P,AvgSin,PROP_12
0,0,2336,25.146513,25.14651,25.146513,0,25.14651,0,25.146513,0,25.14651,0,25.146513,0,25.14651,0,25.146513,0,25.14651,0,25.146513,0,25.14651,0,25.146513,0,25.14651,0,0.0,0.0,0.0,0.0,0.0,0.0,30.6503,0.231
1,0,2337,40.000000,40.00000,40.000000,0,40.00000,0,40.000000,0,40.00000,0,40.000000,0,40.00000,0,40.000000,0,40.00000,0,40.000000,0,40.00000,0,40.000000,0,40.00000,0,0.0,0.0,0.0,0.0,0.0,0.0,25.7654,0.213
2,0,2338,35.799999,35.80000,35.799999,0,35.80000,0,35.799999,0,35.80000,0,35.799999,0,35.80000,0,35.799999,0,35.80000,0,35.799999,0,35.80000,0,35.799999,0,35.80000,0,0.0,0.0,0.0,0.0,0.0,0.0,26.4273,0.196
3,0,2339,44.693333,44.69333,44.693333,0,44.69333,0,44.693333,0,44.69333,0,44.693333,0,44.69333,0,44.693333,0,44.69333,0,44.693333,0,44.69333,0,44.693333,0,44.69333,0,0.0,0.0,0.0,0.0,0.0,0.0,25.8114,0.179
4,0,2340,45.193333,45.19333,45.193333,0,45.19333,0,45.193333,0,45.19333,0,45.193333,0,45.19333,0,45.193333,0,45.19333,0,45.193333,0,45.19333,0,45.193333,0,45.19333,0,0.0,0.0,0.0,0.0,0.0,0.0,25.5051,0.176
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4250,0,390969,7.000000,7.00000,7.000000,0,7.00000,0,7.000000,0,7.00000,0,7.000000,0,7.00000,0,7.000000,0,7.00000,0,7.000000,0,7.00000,0,7.000000,0,7.00000,0,0.0,0.0,0.0,0.0,0.0,0.0,15.1952,0.125
4251,0,390970,7.457831,7.45783,7.457831,0,7.45783,0,7.457831,0,7.45783,0,7.457831,0,7.45783,0,7.457831,0,7.45783,0,7.457831,0,7.45783,0,7.457831,0,7.45783,0,0.0,0.0,0.0,0.0,0.0,0.0,15.1952,0.119
4252,0,390971,8.333333,8.33333,8.333333,0,8.33333,0,8.333333,0,8.33333,0,8.333333,0,8.33333,0,8.333333,0,8.33333,0,8.333333,0,8.33333,0,8.333333,0,8.33333,0,0.0,0.0,0.0,0.0,0.0,0.0,15.1952,0.163
4253,0,394849,122.400002,122.40000,122.400002,0,122.40000,0,122.400002,0,122.40000,0,122.400002,0,122.40000,0,122.400002,0,122.40000,0,122.400002,0,122.40000,0,122.400002,0,122.40000,0,0.0,0.0,0.0,0.0,0.0,0.0,46.7659,0.000


In [69]:
import pandas as pd
df=df_nuevo
# Suponiendo que su DataFrame se llama "df"
diff_1 = ((df.iloc[:, 4] - df.iloc[:, 6]) / df.iloc[:, 6]) * 100
diff_2 = ((df.iloc[:, 8] - df.iloc[:, 10]) / df.iloc[:, 10]) * 100
diff_3 = ((df.iloc[:, 12] - df.iloc[:, 14]) / df.iloc[:, 14]) * 100
diff_4 = ((df.iloc[:, 16] - df.iloc[:, 18]) / df.iloc[:, 18]) * 100
diff_5 = ((df.iloc[:, 20] - df.iloc[:, 22]) / df.iloc[:, 22]) * 100
diff_6 = ((df.iloc[:, 24] - df.iloc[:, 26]) / df.iloc[:, 26]) * 100



df.insert(loc=8, column='D1', value=diff_1)
df.insert(loc=12, column='D2', value=diff_2)
df.insert(loc=16, column='D3', value=diff_3)
df.insert(loc=20, column='D4', value=diff_4)
df.insert(loc=24, column='D5', value=diff_5)
df.insert(loc=28, column='D6', value=diff_6)







In [70]:
df

,EVAL_P,ID,Ag_ppm_BESTEL,VALUE,CAP1_P,CPD1_P,CAPVAL1,CAPPED1,D1,CAP2_P,CPD2_P,CAPVAL2,D2,CAPPED2,CAP3_P,CPD3_P,D3,CAPVAL3,CAPPED3,CAP4_P,D4,CPD4_P,CAPVAL4,CAPPED4,D5,CAP5_P,CPD5_P,CAPVAL5,D6,CAPPED5,CAP6_P,CPD6_P,CAPVAL6,CAPPED6,SUM_SN_P,SUM_CN_P,AVG_SN_P,AVG_CN_P,PROP_SN_P,PROP_CN_P,AvgSin,PROP_12
0,0,2336,25.146513,25.14651,25.146513,0,25.14651,0,0.000012,25.146513,0,25.14651,0.000012,0,25.146513,0,0.000012,25.14651,0,25.146513,0.000012,0,25.14651,0,0.000012,25.146513,0,25.14651,0.000012,0,25.146513,0,25.14651,0,0.0,0.0,0.0,0.0,0.0,0.0,30.6503,0.231
1,0,2337,40.000000,40.00000,40.000000,0,40.00000,0,0.000000,40.000000,0,40.00000,0.000000,0,40.000000,0,0.000000,40.00000,0,40.000000,0.000000,0,40.00000,0,0.000000,40.000000,0,40.00000,0.000000,0,40.000000,0,40.00000,0,0.0,0.0,0.0,0.0,0.0,0.0,25.7654,0.213
2,0,2338,35.799999,35.80000,35.799999,0,35.80000,0,-0.000003,35.799999,0,35.80000,-0.000003,0,35.799999,0,-0.000003,35.80000,0,35.799999,-0.000003,0,35.80000,0,-0.000003,35.799999,0,35.80000,-0.000003,0,35.799999,0,35.80000,0,0.0,0.0,0.0,0.0,0.0,0.0,26.4273,0.196
3,0,2339,44.693333,44.69333,44.693333,0,44.69333,0,0.000007,44.693333,0,44.69333,0.000007,0,44.693333,0,0.000007,44.69333,0,44.693333,0.000007,0,44.69333,0,0.000007,44.693333,0,44.69333,0.000007,0,44.693333,0,44.69333,0,0.0,0.0,0.0,0.0,0.0,0.0,25.8114,0.179
4,0,2340,45.193333,45.19333,45.193333,0,45.19333,0,0.000007,45.193333,0,45.19333,0.000007,0,45.193333,0,0.000007,45.19333,0,45.193333,0.000007,0,45.19333,0,0.000007,45.193333,0,45.19333,0.000007,0,45.193333,0,45.19333,0,0.0,0.0,0.0,0.0,0.0,0.0,25.5051,0.176
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4250,0,390969,7.000000,7.00000,7.000000,0,7.00000,0,0.000000,7.000000,0,7.00000,0.000000,0,7.000000,0,0.000000,7.00000,0,7.000000,0.000000,0,7.00000,0,0.000000,7.000000,0,7.00000,0.000000,0,7.000000,0,7.00000,0,0.0,0.0,0.0,0.0,0.0,0.0,15.1952,0.125
4251,0,390970,7.457831,7.45783,7.457831,0,7.45783,0,0.000013,7.457831,0,7.45783,0.000013,0,7.457831,0,0.000013,7.45783,0,7.457831,0.000013,0,7.45783,0,0.000013,7.457831,0,7.45783,0.000013,0,7.457831,0,7.45783,0,0.0,0.0,0.0,0.0,0.0,0.0,15.1952,0.119
4252,0,390971,8.333333,8.33333,8.333333,0,8.33333,0,0.000036,8.333333,0,8.33333,0.000036,0,8.333333,0,0.000036,8.33333,0,8.333333,0.000036,0,8.33333,0,0.000036,8.333333,0,8.33333,0.000036,0,8.333333,0,8.33333,0,0.0,0.0,0.0,0.0,0.0,0.0,15.1952,0.163
4253,0,394849,122.400002,122.40000,122.400002,0,122.40000,0,0.000002,122.400002,0,122.40000,0.000002,0,122.400002,0,0.000002,122.40000,0,122.400002,0.000002,0,122.40000,0,0.000002,122.400002,0,122.40000,0.000002,0,122.400002,0,122.40000,0,0.0,0.0,0.0,0.0,0.0,0.0,46.7659,0.000


In [71]:
df_x = df[df["EVAL_P"] == 99]
df_x

,EVAL_P,ID,Ag_ppm_BESTEL,VALUE,CAP1_P,CPD1_P,CAPVAL1,CAPPED1,D1,CAP2_P,CPD2_P,CAPVAL2,D2,CAPPED2,CAP3_P,CPD3_P,D3,CAPVAL3,CAPPED3,CAP4_P,D4,CPD4_P,CAPVAL4,CAPPED4,D5,CAP5_P,CPD5_P,CAPVAL5,D6,CAPPED5,CAP6_P,CPD6_P,CAPVAL6,CAPPED6,SUM_SN_P,SUM_CN_P,AVG_SN_P,AVG_CN_P,PROP_SN_P,PROP_CN_P,AvgSin,PROP_12
73,99,6586,0.828571,0.82857,0.828571,0,0.82857,0,0.000121,0.828571,0,0.82857,0.000121,0,0.828571,0,0.000121,0.82857,0,0.828571,0.000121,0,0.82857,0,0.000121,0.828571,0,0.82857,0.000121,0,0.828571,0,0.82857,0,226.805611,227.634182,11.340281,10.839723,0.003653,0.003640,13.7859,0.095
74,99,6587,2.000000,2.00000,2.000000,0,2.00000,0,0.000000,2.000000,0,2.00000,0.000000,0,2.000000,0,0.000000,2.00000,0,2.000000,0.000000,0,2.00000,0,0.000000,2.000000,0,2.00000,0.000000,0,2.000000,0,2.00000,0,226.805611,228.805611,11.340281,10.895505,0.008818,0.008741,13.7859,0.095
75,99,6588,10.200000,10.20000,10.200000,0,10.20000,0,0.000000,10.200000,0,10.20000,0.000000,0,10.200000,0,0.000000,10.20000,0,10.200000,0.000000,0,10.20000,0,0.000000,10.200000,0,10.20000,0.000000,0,10.200000,0,10.20000,0,226.805611,237.005611,11.340281,11.285981,0.044972,0.043037,13.7859,0.095
76,99,6589,7.300000,7.30000,7.300000,0,7.30000,0,0.000000,7.300000,0,7.30000,0.000000,0,7.300000,0,0.000000,7.30000,0,7.300000,0.000000,0,7.30000,0,0.000000,7.300000,0,7.30000,0.000000,0,7.300000,0,7.30000,0,231.957126,239.257126,11.597856,11.393196,0.031471,0.030511,13.7859,0.095
77,99,6590,35.708694,35.70869,4.686572,9,35.70869,0,-86.875543,5.579289,9,35.70869,-84.375543,0,35.708694,0,0.000011,35.70869,0,35.708694,0.000011,0,35.70869,0,0.000011,35.708694,0,35.70869,0.000011,0,35.708694,0,35.70869,0,943.116209,978.824903,47.155810,46.610710,0.037862,0.036481,13.7859,0.143
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4233,99,386872,74.000000,74.00000,9.564257,9,69.44965,1,-86.228502,11.414257,9,74.00000,-84.575328,0,13.264257,9,-82.075328,74.00000,0,15.114257,-79.575328,9,74.00000,0,-77.075328,16.964257,9,74.00000,0.000000,0,74.000000,0,74.00000,0,266.654761,340.654761,0.246219,0.314257,0.277512,0.217229,21.7030,0.955
4234,99,386873,35.000000,35.00000,4.653279,9,35.00000,0,-86.704916,35.000000,0,35.00000,0.000000,0,35.000000,0,0.000000,35.00000,0,35.000000,0.000000,0,35.00000,0,0.000000,35.000000,0,35.00000,0.000000,0,35.000000,0,35.00000,0,266.654761,301.654761,0.246219,0.278279,0.131256,0.116027,14.2111,0.186
4235,99,386874,17.799999,17.80000,17.799999,0,17.80000,0,-0.000006,17.799999,0,17.80000,-0.000006,0,17.799999,0,-0.000006,17.80000,0,17.799999,-0.000006,0,17.80000,0,-0.000006,17.799999,0,17.80000,-0.000006,0,17.799999,0,17.80000,0,266.654761,284.454760,0.246219,0.262412,0.066753,0.062576,14.5349,0.186
4236,99,386875,13.000000,13.00000,13.000000,0,13.00000,0,0.000000,13.000000,0,13.00000,0.000000,0,13.000000,0,0.000000,13.00000,0,13.000000,0.000000,0,13.00000,0,0.000000,13.000000,0,13.00000,0.000000,0,13.000000,0,13.00000,0,266.654761,279.654761,0.246219,0.257984,0.048752,0.046486,14.9206,0.186


In [72]:
df_x.to_excel("filtered_andMerged.xlsx", index=False)

In [23]:
df_CLAG_MR_2022.to_excel("Mx.xlsx")

In [11]:
df_CLAG_MR_2022

,mid_x,mid_y,mid_z,Ag_ppm_BESTEL,NBKRT,AG_SUBZONAS,HOLEID,x_rot,y_rot,z_rot,CAP1_P,CPD1_P,CAP2_P,CPD2_P,CAP3_P,CPD3_P,CAP4_P,CPD4_P,CAP5_P,CPD5_P,CAP6_P,CPD6_P,EVAL_P,NEXP_P,LEN_NT_P,LEN_TO_P,SUM_SN_P,SUM_CN_P,AVG_SN_P,AVG_CN_P,PROP_SN_P,PROP_CN_P
COUNT,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2336,4072.0,4850.0,4200.0,25.0,32,42,A019,0.0,0.0,0,25.0,0,25.0,0,25.0,0,25.0,0,25.0,0,25.0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0
2337,4074.0,4849.0,4197.0,40.0,31,42,A019,0.0,0.0,0,40.0,0,40.0,0,40.0,0,40.0,0,40.0,0,40.0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0
2338,4075.0,4848.0,4194.0,36.0,31,42,A019,0.0,0.0,0,36.0,0,36.0,0,36.0,0,36.0,0,36.0,0,36.0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0
2339,4076.0,4846.0,4192.0,45.0,31,42,A019,0.0,0.0,0,45.0,0,45.0,0,45.0,0,45.0,0,45.0,0,45.0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0
2340,4077.0,4845.0,4189.0,45.0,31,42,A019,0.0,0.0,0,45.0,0,45.0,0,45.0,0,45.0,0,45.0,0,45.0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390969,4235.0,5191.0,4185.0,7.0,31,42,A913,0.0,0.0,0,7.0,0,7.0,0,7.0,0,7.0,0,7.0,0,7.0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0
390970,4235.0,5191.0,4182.0,7.0,31,42,A913,0.0,0.0,0,7.0,0,7.0,0,7.0,0,7.0,0,7.0,0,7.0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0
390971,4235.0,5190.0,4178.0,8.0,31,42,A913,0.0,0.0,0,8.0,0,8.0,0,8.0,0,8.0,0,8.0,0,8.0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0


In [16]:
df_CLAG_MR_2022.iloc[:,10:21]

,CAP1_P,CPD1_P,CAP2_P,CPD2_P,CAP3_P,CPD3_P,CAP4_P,CPD4_P,CAP5_P,CPD5_P,CAP6_P
COUNT,,,,,,,,,,,
2336,25.0,0,25.0,0,25.0,0,25.0,0,25.0,0,25.0
2337,40.0,0,40.0,0,40.0,0,40.0,0,40.0,0,40.0
2338,36.0,0,36.0,0,36.0,0,36.0,0,36.0,0,36.0
2339,45.0,0,45.0,0,45.0,0,45.0,0,45.0,0,45.0
2340,45.0,0,45.0,0,45.0,0,45.0,0,45.0,0,45.0
...,...,...,...,...,...,...,...,...,...,...,...
390969,7.0,0,7.0,0,7.0,0,7.0,0,7.0,0,7.0
390970,7.0,0,7.0,0,7.0,0,7.0,0,7.0,0,7.0
390971,8.0,0,8.0,0,8.0,0,8.0,0,8.0,0,8.0
